In [11]:
import requests
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [12]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')

path = 'D:\Documents\code\Scraping/chromedriver.exe'
driver = webdriver.Chrome(executable_path = path, options=chrome_options)

C:\Users\novandra\AppData\Local\Temp\ipykernel_14856\2594541221.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=chrome_options)


In [14]:
judul_artikel, tautan, artikel = [], [], []
for page in range (0,10):
    link_utama = 'https://mediaindonesia.com/search?q=subsidi%20bbm#gsc.tab=0&gsc.q=%22subsidi%20bbm%22&gsc.page={}'.format(page)
    driver.get(link_utama)

In [15]:
    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    mediaind = bs(html,"html.parser")

In [16]:
    for i in mediaind:
        containers = mediaind.findAll('div', class_='gsc-webResult gsc-result')
        for i in  containers:
            ajudul_artikel = i.find(class_='gs-title').find('a')
            bjudul_artikel = ajudul_artikel.get_text()
            judul_artikel.append(bjudul_artikel)
        
        for i in containers:
            url = i.find('a')
            tautan.append(url.get('href'))

In [17]:
#selanjutnya buka setiap link, dan ambil isi artikelnya
for link_artikel in tautan:
    urls = requests.get(link_artikel).content
    laman_mediaind = bs(urls,'html.parser')
    berita_1 = laman_mediaind.find_all('p')

    list_text = []
    for list in berita_1:
        j = list.text
        list_text.append(j)
    full_text = " ".join (list_text)
    artikel.append(full_text)

In [18]:
#buat data frame
list_kolom = ['judul_artikel', 'tautan', 'artikel']
list_data = dict(zip(
            list_kolom,
                (judul_artikel,
                tautan,
                artikel)
))

In [19]:
with open ('media indonesia - transformasi ekonomi.json', 'w') as fp:
    json.dump(list_data, fp)

In [20]:
df = pd.DataFrame(data = list_data)
df.head()

,judul_artikel,tautan,artikel
0,Indonesia Hadapi Banyak Tantangan Pemulihan di...,https://mediaindonesia.com/ekonomi/466763/indo...,ANTARA/MUHAMMAD ADIMAJA\nMenko Perekonomian Ai...
1,"Harga Komoditas Naik, Ekonom : Jangan Bangga S...",https://mediaindonesia.com/ekonomi/466795/harg...,ANTARA FOTO/Aswaddy Hamid\nPekerja memuat miny...
2,Investasi Inklusif untuk Masa Depan Indonesia,https://mediaindonesia.com/kolom-pakar/464741/...,MI/Seno\n \n TAHUN Baru bukan hanya lintasan w...
3,"Indikator Membaik, Pemulihan Ekonomi Diharapka...",https://mediaindonesia.com/ekonomi/464086/indi...,ANTARA\nIlustrasi\n MENTERI Koordinator Bidang...
4,Luhut Janji Percepat Izin Usaha ke Investor di...,https://mediaindonesia.com/ekonomi/459751/luhu...,Antara \nMenko Bidang Kemaritiman dan Investas...


In [21]:
df.to_csv(r'D:\Documents\code\Scraping\media indonesia - transformasi ekonomi2.csv', index = False, header=True)